In [ ]:
#!pip install --upgrade tabula-py
#!pip install camelot-py[pdf] pandas openpyxl
#!pip install pillow
#!pip install pdf2image
#!pip install Spire.PDF
!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (212 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


#Text_Extraction

In [ ]:
#!pip install pymupdf pdfminer.six pypdf2 pdfplumber python-docx
!pip install camelot-py[cv]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.7 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
from docx import Document

def clean_text(text):
    return ''.join(c for c in text if ord(c) in range(32, 127) or ord(c) in [9, 10, 13])

def extract_text_pymupdf(pdf_path, output_docx):
    doc = fitz.open(pdf_path)
    document = Document()

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        clean_txt = clean_text(text)
        document.add_heading(f"Page {page_num + 1}", level=1)
        document.add_paragraph(clean_txt)

    document.save(output_docx)

pdf_path = "/content/amc-annual-report---2019---20.pdf"
output_docx = "PyMuPDF_with_text.docx"
extract_text_pymupdf(pdf_path, output_docx)

In [ ]:
from pdfminer.high_level import extract_text
from docx import Document

def clean_text(text):
    return ''.join(c for c in text if ord(c) in range(32, 127) or ord(c) in [9, 10, 13])

def extract_text_pdfminer(pdf_path, output_docx):
    text = extract_text(pdf_path)
    clean_txt = clean_text(text)
    document = Document()
    document.add_paragraph(clean_txt)
    document.save(output_docx)

pdf_path = "/content/amc-annual-report---2019---20.pdf"
output_docx = "PDFMiner_with_text.docx"
extract_text_pdfminer(pdf_path, output_docx)

In [ ]:
import PyPDF2
from docx import Document

def clean_text(text):
    return ''.join(c for c in text if ord(c) in range(32, 127) or ord(c) in [9, 10, 13])

def extract_text_pypdf2(pdf_path, output_docx):
    document = Document()
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            clean_txt = clean_text(text)
            document.add_heading(f"Page {page_num + 1}", level=1)
            document.add_paragraph(clean_txt)

    document.save(output_docx)

pdf_path = "/content/amc-annual-report---2019---20.pdf"
output_docx = "PyPDF2_with_text.docx"
extract_text_pypdf2(pdf_path, output_docx)

In [ ]:
import pdfplumber
from docx import Document

def clean_text(text):
    return ''.join(c for c in text if ord(c) in range(32, 127) or ord(c) in [9, 10, 13])

def extract_text_pdfplumber(pdf_path, output_docx):
    document = Document()
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()
            clean_txt = clean_text(text)
            document.add_heading(f"Page {page_num + 1}", level=1)
            document.add_paragraph(clean_txt)

    document.save(output_docx)

pdf_path = "/content/amc-annual-report---2019---20.pdf"
output_docx = "pdfplumber_with_text.docx"
extract_text_pdfplumber(pdf_path, output_docx)

#Table_Extraction

In [ ]:
import pdfplumber
from docx import Document

def extract_tables_pdfplumber(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            if extracted_tables:
                tables.extend(extracted_tables)
    return tables

def save_tables_to_word(tables, output_docx):
    doc = Document()
    for i, table in enumerate(tables):
        if table:  # Check if the table is not empty
            table_doc = doc.add_table(rows=len(table), cols=len(table[0]))
            for row_idx, row in enumerate(table):
                for col_idx, cell in enumerate(row):
                    table_doc.cell(row_idx, col_idx).text = str(cell)  # Ensure cell content is string
            doc.add_paragraph("\n")  # Add a new line between tables
    doc.save(output_docx)

# Example usage
pdf_path = "/content/amc-annual-report---2020---21.pdf"
tables = extract_tables_pdfplumber(pdf_path)
output_docx = "pdfplumber_tables_extracted.docx"
save_tables_to_word(tables, output_docx)


In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from docx import Document
from docx.shared import Inches

def save_tables_to_word(tables, output_docx, output_folder):
    if not tables:
        print("No tables found in the PDF.")
        return

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    doc = Document()

    # Add each table to the Word document
    for index, table in enumerate(tables):
        table_filename = os.path.join(output_folder, f"table_{index + 1}.png")

        if table.empty:
            print(f"Skipping empty table {index + 1}")
            continue

        # Convert the table to an image using matplotlib
        plt.figure(figsize=(10, 6))  # Adjust size as needed
        ax = plt.subplot(111, frame_on=False)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        plt.table(cellText=table.values, colLabels=table.columns, loc='center')
        plt.savefig(table_filename, bbox_inches='tight', pad_inches=0)
        plt.close()

        # Insert the image into the Word document
        doc.add_picture(table_filename, width=Inches(5))  # Adjust width as needed
        doc.add_paragraph()  # Add space between tables

    # Save the Word document
    doc.save(output_docx)

# Example usage
pdf_path = "/content/amc-annual-report---2020---21.pdf"
tables = extract_tables_tabula(pdf_path)
output_docx = "tables_from_pdf.docx"
output_folder = "table_images_matplotlib"
save_tables_to_word(tables, output_docx, output_folder)


Jun 06, 2024 12:50:26 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jun 06, 2024 12:50:29 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jun 06, 2024 12:51:14 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile

) missing from current font.
  plt.savefig(table_filename, bbox_inches='tight', pad_inches=0)


Skipping empty table 9
Skipping empty table 25
Skipping empty table 31
Skipping empty table 47
Skipping empty table 50
Skipping empty table 52
Skipping empty table 53
Skipping empty table 55
Skipping empty table 65
Skipping empty table 69
Skipping empty table 70
Skipping empty table 73
Skipping empty table 74
Skipping empty table 78
Skipping empty table 79
Skipping empty table 82
Skipping empty table 86
Skipping empty table 91
Skipping empty table 92
Skipping empty table 94
Skipping empty table 96
Skipping empty table 99
Skipping empty table 104
Skipping empty table 109
Skipping empty table 110
Skipping empty table 113
Skipping empty table 117
Skipping empty table 120
Skipping empty table 121
Skipping empty table 123
Skipping empty table 124
Skipping empty table 128
Skipping empty table 135
Skipping empty table 138
Skipping empty table 151
Skipping empty table 155
Skipping empty table 156
Skipping empty table 159
Skipping empty table 165
Skipping empty table 170
Skipping empty table 17

#image extractiion

In [ ]:
import io
from PIL import Image
from docx import Document
from docx.shared import Inches
import fitz

def extract_images_pymupdf(pdf_path):
    images = []
    doc = fitz.open(pdf_path)
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        image_list = page.get_images(full=True)
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
    return images

def save_images_to_word(images, output_docx):
    doc = Document()
    doc.add_heading('Images extracted with PyMuPDF', level=1)
    for i, image in enumerate(images):
        # Convert the image to RGB mode to avoid the 'read' attribute error
        image = image.convert("RGB")
        # Add the image to the document
        img_io = io.BytesIO()
        image.save(img_io, format='JPEG')
        img_io.seek(0)
        doc.add_picture(img_io, width=Inches(5))
        doc.add_paragraph(f'Image {i+1}')
    doc.save(output_docx)

# Example usage
pdf_path = "/content/amc-annual-report---2020---21.pdf"
images_pymupdf = extract_images_pymupdf(pdf_path)
output_docx_pymupdf = "images_pymupdf.docx"
save_images_to_word(images_pymupdf, output_docx_pymupdf)


In [ ]:
from pdf2image import convert_from_path
import os

# Path to the PDF file
pdf_path = '/content/amc-annual-report---2020---21.pdf'

# Directory to save extracted images
output_folder = '/content/pdf2image_extraction'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Extract images from the PDF
images = convert_from_path(pdf_path)

# Save the extracted images
for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'Image_{i}.png')
    image.save(image_path, 'PNG')

In [ ]:
import os

# Create a directory to save images if it doesn't exist
os.makedirs('pdf2image_extraction', exist_ok=True)

In [ ]:
!zip -r pdf2image_extraction.zip pdf2image_extraction/

  adding: pdf2image_extraction/ (stored 0%)
  adding: pdf2image_extraction/Image_98.png (deflated 9%)
  adding: pdf2image_extraction/Image_14.png (deflated 7%)
  adding: pdf2image_extraction/Image_116.png (deflated 9%)
  adding: pdf2image_extraction/Image_83.png (deflated 7%)
  adding: pdf2image_extraction/Image_21.png (deflated 7%)
  adding: pdf2image_extraction/Image_55.png (deflated 8%)
  adding: pdf2image_extraction/Image_103.png (deflated 8%)
  adding: pdf2image_extraction/Image_10.png (deflated 8%)
  adding: pdf2image_extraction/Image_5.png (deflated 13%)
  adding: pdf2image_extraction/Image_118.png (deflated 9%)
  adding: pdf2image_extraction/Image_6.png (deflated 6%)
  adding: pdf2image_extraction/Image_140.png (deflated 7%)
  adding: pdf2image_extraction/Image_40.png (deflated 7%)
  adding: pdf2image_extraction/Image_77.png (deflated 7%)
  adding: pdf2image_extraction/Image_95.png (deflated 7%)
  adding: pdf2image_extraction/Image_119.png (deflated 8%)
  adding: pdf2image_extr

In [ ]:
from google.colab import files

# Download the zip file
files.download('pdf2image_extraction.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>